[如何用python重复一篇Nature子刊观测数据的部分结果](https://zhuanlan.zhihu.com/p/409488179?utm_source=wechat_session&utm_medium=social&utm_oi=557157888095735808&utm_campaign=shareopn&s_r=0如何用python重复一篇Nature子刊观测数据的部分结果)

[Cosmological constraints from the Hubble diagram of quasars at high redshifts, G. Risaliti  and E. Lusso(2019)](https://www.nature.com/articles/s41550-018-0657-z)

这是一篇关于类星体（Quasar）观测数据给出宇宙学模型检验的文章，发表在Nature Astronomy.

简单介绍这里的物理内容。

参考arXiv:2103.16032, arXiv:2105.04992

In [1]:
import matplotlib.pyplot as plt
from scipy import integrate
import math
import sys, os
#sys.path.insert(0,os.path.realpath(os.path.join(os.getcwd(),'..')))
#from sympy import Symbol
from sympy import *
from scipy.integrate import simps
from scipy.optimize import root
from scipy.optimize import minimize

In [ ]:
file_name = 'quasar_data_RL.txt'
data=np.loadtxt(file_name, skiprows=0, usecols = (0,3,4,5), dtype=np.float32)
data

zx=data[:,0]
FUV=data[:,1]
FX=data[:,2]
eFX=data[:,3]


ss=np.argsort(zx)
FUVsort=FUV[ss]
FXsort=FX[ss]
eFXsort=eFX[ss]
zxsort=zx[ss]

from mpl_toolkits.mplot3d import Axes3D
plt.rcParams['figure.figsize'] = (8.0, 8.0)
fig = plt.figure('3D scatter plot')
ax = fig.add_subplot(111, projection= '3d')  #3d图需要加projection='3d'
cValue = ['r','y','g','b','r','y','g','b','r'] 
# 用不同颜色、形状的图标区分两组数据
ax.scatter(zx,FUV,FX, c='r' , marker='.')
ax.set_xlabel('z')
ax.set_ylabel('$log_{10}F_{UV}$',fontsize='10')
ax.set_zlabel('$log_{10}F_{X}$',fontsize='10')
ax.set_title('Quasar',fontsize='10')
plt.savefig('QuasarDataPlot1.pdf')
plt.show()

In [ ]:
def CutbinbyZ(data0,ztable,z1,z2):
    data1=data0[np.where((ztable>=z1)&(ztable<z2))]
    return data1

z1=0
z2=1.4
FUV14= CutbinbyZ(FUVsort,zxsort,z1,z2)
FX14= CutbinbyZ(FXsort,zxsort,z1,z2)
eFX14= CutbinbyZ(eFXsort,zxsort,z1,z2)
zx14= CutbinbyZ(zxsort,zxsort,z1,z2)

z1=1.4
z2=10
FUV0= CutbinbyZ(FUVsort,zxsort,z1,z2)
FX0= CutbinbyZ(FXsort,zxsort,z1,z2)
eFX0= CutbinbyZ(eFXsort,zxsort,z1,z2)
zx0= CutbinbyZ(zxsort,zxsort,z1,z2)

from mpl_toolkits.mplot3d import Axes3D
plt.rcParams['figure.figsize'] = (8.0, 8.0)
fig = plt.figure('3D scatter plot')
ax = fig.add_subplot(111, projection= '3d')  #3d图需要加projection='3d'
cValue = ['blue','y','g','b','r','y','g','b','r'] 
# 用不同颜色、形状的图标区分两组数据
ax.scatter(zx0,FUV0,FX0, c='y' , marker='.')
ax.scatter(zx14,FUV14,FX14, c='blue' , marker='.')
ax.set_xlabel('z')
ax.set_ylabel('$\log_{10}F_{UV}$',fontsize='30')
ax.set_zlabel('$\log_{10}F_{X}$',fontsize='30')
ax.set_title('Quasar',fontsize='30')
plt.savefig('QuasarDataPlot2.pdf')
plt.show()

In [ ]:
plt.rcParams['figure.figsize'] = (10.0, 8.0)
plt.errorbar(FUV14,FX14,eFX14, fmt=".r", capsize=2, label= 'QSO(0<z<1.4)',ecolor='green')
plt.errorbar(FUV0,FX0,eFX0, fmt=".b", capsize=2, label= 'QSO(z>1.4)',ecolor='blue')
plt.legend(fontsize=14)
plt.xlim()
plt.ylim()
plt.xlabel('$log_{10}F_{UV}$',fontsize='30')
plt.ylabel('$log_{10}F_{X}$',fontsize='30')
plt.title('Quasar',fontsize='30')
plt.savefig('QuasarF.pdf')
plt.show()

In [ ]:
H0=0.7000
PI=np.pi
def log_likelihoodQ(theta):
    gamma,beta,delta,a2,a3 = theta
    z=zxsort
    x=FUVsort
    y=FXsort
    yerr=eFXsort
    xx=np.log10(z+1)+a2*np.log10(z+1)**2+a3*np.log10(z+1)**3
    yy=np.log(10)*3e5/(100.0*H0)*xx
    logD=np.log10(yy)            
    model=gamma*x+(2*gamma-2)*(logD+24.48935370050942)+1*(gamma-1)*np.log10(4*PI )+beta
    sigamaUV=0
    sigma2 = yerr ** 2+gamma**2*sigamaUV**2+delta**2
    xi=(y - model) ** 2 / sigma2+np.log(2*PI*sigma2 )
    ChiS=np.sum(xi)
    return   -ChiS

def log_priorQ(theta):
    gamma,beta,delta,a2,a3= theta
    if 0.0 < gamma < 2.0 and -30.0 <beta <30.0\
    and 0.0 < delta < 2.0 and -5 <a2 < 10.0\
    and -5 < a3 < 10.0:
        return -0
    return -np.inf

def log_probabilityQ(theta):
    lp = log_priorQ(theta)
    if not np.isfinite(lp):
        return -np.inf
    return lp + log_likelihoodQ(theta)

In [ ]:
plt.rcParams['figure.figsize'] = (10.0, 8.0)
file_name = 'lcparam_full_long_zhel.txt'
data=np.loadtxt(file_name, skiprows=1, usecols = (1,4,5), dtype=np.float32)
xS=data[:,0]
yS=data[:,1]
yerrS=data[:,2]
plt.errorbar(xS, yS, yerrS, fmt=".b", capsize=0, label="SNIa",ecolor='orange')
plt.legend(fontsize=14)
plt.xlim(0, 2.5)
plt.xlabel("z")
plt.ylabel("Apparent Magnitude $m$");
plt.savefig('SNIa.pdf')
plt.show()


In [ ]:
def log_likelihoodS(theta):
    Mb,a2,a3 = theta
    z=xS
    y=yS
    yerr=yerrS
    xx=np.log10(z+1)+a2*np.log10(z+1)**2+a3*np.log10(z+1)**3
    yy=np.log(10)*3e5/(100.0*H0)*xx
    modelS=25+5*np.log10(yy)+Mb
    sigma2S = yerr ** 2
    xiS=(y - modelS) ** 2 / sigma2S
    ChiS=np.sum(xiS)
    return   -ChiS
def log_priorS(theta):
    Mb,a2,a3= theta
    if -30 < Mb < 0 and -5 <a2 < 10.0\
    and -5 < a3 < 10.0:
        return -0
    return -np.inf

def log_probabilityS(theta):
    lp = log_priorS(theta)
    if not np.isfinite(lp):
        return -np.inf
    return lp + log_likelihoodS(theta)

In [ ]:
#theta= [gamma,beta,delta,Mb,a2,a3]
def log_likelihood(theta):
    gamma,beta,delta,Mb,a2,a3=theta
    thetaQ=gamma,beta,delta,a2,a3
    thetaS=Mb,a2,a3 
    lkh=log_likelihoodS(thetaS)+log_likelihoodQ(thetaQ)
    return lkh

def log_probability(theta):
    gamma,beta,delta,Mb,a2,a3=theta
    thetaQ=gamma,beta,delta,a2,a3
    thetaS=Mb,a2,a3 
    lkh=log_probabilityS(thetaS)+log_probabilityQ(thetaQ)
    return lkh

In [ ]:
# Bounds on variables for Nelder-Mead, L-BFGS-B, TNC, SLSQP, Powell, and trust-constr methods. 
dim=6
theta0=[0.6,7,0.23,-19, 3.0,3.0]
s0=1
s1=5
s2=1
s3=4
s4=4
s5=4
# gamma, beta,delta,a2,a3= theta

bounds=((theta0[0]-s0,theta0[0]+s0),(theta0[1]-s1,theta0[1]+s1),(theta0[2]-s2,theta0[2]+s2),
        (theta0[3]-s3,theta0[3]+s3),(theta0[4]-s4,theta0[4]+s4),(theta0[5]-s5,theta0[5]+s5))
np.random.seed(42)
nll = lambda *args: -log_likelihood(*args)
initial = np.array(theta0) + 0.001 * np.random.randn(dim)
soln = minimize(nll, initial, args=(),bounds=bounds, method='Powell')
sol= soln.x

print('Best-fit result is\n %s \nThe minima of chi square is %s\n'%(sol,log_likelihood(sol)))

thetam=[ 0.63126393,   7.31041219,   0.23628922, -19.37243797,   3.1449026, 3.28581407]
print('Median value from MCMC result is \n %s \
      \nThe chi square for median value is %s'%(thetam,log_likelihood(thetam)))

In [ ]:
sol=theta0
import emcee
pos = sol + 1e-6 * np.random.randn(32, dim)
nwalkers, ndim = pos.shape
sampler = emcee.EnsembleSampler(nwalkers, ndim, log_probability, args=())

sampler.run_mcmc(pos,3000, progress=True);


In [ ]:
samples = sampler.get_chain()
flat_samples = sampler.get_chain(discard=1000, thin=10, flat=True)  

from getdist import plots, MCSamples
import getdist

labels= ['\gamma', r'\beta', '\delta','M_B','a_2','a_3']
names=  [ '\gamma', r'\beta', '\delta','M_B','a_2','a_3']
samples0 = MCSamples(samples=flat_samples,names =names, labels = labels)
g = plots.get_subplot_plotter()
g.settings.alpha_filled_add=0.4
g.settings.title_limit_fontsize = 14
g.triangle_plot(samples0 , filled=True, legend_labels=['Quasars(QSO)\narXiv:2004.09979\narXiv:2103.07139', 'Simulation 2'], 
    legend_loc='upper right',title_limit=1)

labels=labels
plt.savefig('QSQ_SNe.pdf')

In [ ]:
g = plots.get_subplot_plotter()
g.settings.alpha_filled_add=0.4
g.settings.title_limit_fontsize = 14
samples0.updateSettings({'contours': [0.68, 0.95, 0.997,0.9999]})
g.settings.num_plot_contours = 4

g.triangle_plot([samples0], ['a_2', 'a_3'] , filled=True,
                legend_labels=['$a_2$-$a_3$', 'Simulation 2'], 
                legend_loc='upper right',title_limit=1,
                colors=['orange'],
                line_args=[{'ls':'--', 'color':'blue'}])

plt.savefig('a2a32.pdf')